# HW3 Support Vector Machine



---

#### Name: 吳原博
#### Student ID: 0816004

---

## 助教好 😀
> 程式碼的解釋會放在每個程式碼區塊上方的 markdown 裡，謝謝!

In [118]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import *
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
%matplotlib inline

# Variable
用於 parameter search 的參數，這是為了 Question 而調的比較大，實際上當初我做了好幾次調整才得到現在的結果

In [119]:
param_pol = {'degree':[2,5,10],'gamma':[0.01,0.16,0.5],'coef0':[0.01,0.0015,0.1]}
param_rbf = {'gamma':[0.01,0.16,0.5]}

# Data Processing
這次資料比較單純，以 csv 讀取後分好 label 和 data 再切成 7:3

In [120]:
data = pd.read_csv('ionosphere.data',header=None)
Y = data[34]
X = data.drop(34,axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3)

# Support Vector Machine

### Parameter Search
我使用 skleanr 的 GridSearchCV 做參數調整，以 cv=5 來做到 5-fold validation  
如下面的輸出，兩行分別為 polynomial 和 RBF 的最佳參數

In [121]:
clf_pol = GridSearchCV(SVC(),param_pol,cv=5)
clf_pol.fit(X_train,Y_train)
print(clf_pol.best_params_)
pol_best = clf_pol.best_params_
clf_rbf = GridSearchCV(SVC(),param_rbf,cv=5)
clf_rbf.fit(X_train,Y_train)
print(clf_rbf.best_params_)
rbf_best = clf_rbf.best_params_

{'coef0': 0.01, 'degree': 2, 'gamma': 0.16}
{'gamma': 0.16}


### Model Training && Results
SVC 模型一樣用 sklearn 完成，並且使用前面得到的參數建模型  
訓練的結果以表格呈現在輸出欄位

In [122]:
svc_lin = SVC(kernel='linear')
svc_pol = SVC(kernel='poly',degree=pol_best['degree'],gamma=pol_best['gamma'],coef0=pol_best['coef0'])
svc_rbf = SVC(kernel='rbf',gamma=rbf_best['gamma'])
svc_lin.fit(X_train,Y_train)
svc_pol.fit(X_train,Y_train)
svc_rbf.fit(X_train,Y_train)
result_lin = svc_lin.predict(X_test)
result_pol = svc_pol.predict(X_test)
result_rbf = svc_rbf.predict(X_test)
matrix_lin = confusion_matrix(Y_test,result_lin,labels=['g','b'])
matrix_pol = confusion_matrix(Y_test,result_pol,labels=['g','b'])
matrix_rbf = confusion_matrix(Y_test,result_rbf,labels=['g','b'])
acc_lin = accuracy_score(Y_test,result_lin)
acc_pol = accuracy_score(Y_test,result_pol)
acc_rbf = accuracy_score(Y_test,result_rbf)
rec_lin = recall_score(Y_test,result_lin,average=None,labels=['g','b'])
rec_pol = recall_score(Y_test,result_pol,average=None,labels=['g','b'])
rec_rbf = recall_score(Y_test,result_rbf,average=None,labels=['g','b'])
pre_lin = precision_score(Y_test,result_lin,average=None,labels=['g','b'])
pre_pol = precision_score(Y_test,result_pol,average=None,labels=['g','b'])
pre_rbf = precision_score(Y_test,result_rbf,average=None,labels=['g','b'])
print('confusion matrics:')
print('  Linear')
form = pd.DataFrame(matrix_lin,columns=['g predicted','b predicted'],index=['g','b'])
display(form)
print('  Polynomial')
form = pd.DataFrame(matrix_pol,columns=['g predicted','b predicted'],index=['g','b'])
display(form)
print('  RBF')
form = pd.DataFrame(matrix_rbf,columns=['g predicted','b predicted'],index=['g','b'])
display(form)
print('Scores')
temp = [[acc_lin,acc_pol,acc_rbf],[rec_lin[0],rec_pol[0],rec_rbf[0]],[rec_lin[1],rec_pol[1],rec_rbf[1]],[pre_lin[0],pre_pol[0],pre_rbf[0]],[pre_lin[1],pre_pol[1],pre_rbf[1]]]
form = pd.DataFrame(temp,columns=['linear','polynomial','RBF'],index=['accuracy','recall (pos = g)','recall (pos = b)','precision (pos = g)','precision (pos = b)'])
display(form)

confusion matrics:
  Linear


,g predicted,b predicted
g,64,1
b,10,31


  Polynomial


,g predicted,b predicted
g,64,1
b,6,35


  RBF


,g predicted,b predicted
g,64,1
b,4,37


Scores


,linear,polynomial,RBF
accuracy,0.896226,0.933962,0.952830
recall (pos = g),0.984615,0.984615,0.984615
recall (pos = b),0.756098,0.853659,0.902439
precision (pos = g),0.864865,0.914286,0.941176
precision (pos = b),0.968750,0.972222,0.973684


# Questions
為了讓結果比較明顯，我將參數的間距調得比較大，現在使用的參數其實是經過我幾次微調後的結果

In [125]:
print('Linear')
temp = [clf_pol.cv_results_['mean_test_score'],[i['coef0'] for i in clf_pol.cv_results_['params']],[i['degree'] for i in clf_pol.cv_results_['params']],[i['gamma'] for i in clf_pol.cv_results_['params']]]
temp = np.array(temp).transpose(1,0).tolist()
form = pd.DataFrame(temp,columns=['score','coef0','degree','gamma'])
display(form)
print('RBF')
temp = [clf_rbf.cv_results_['mean_test_score'],[i['gamma'] for i in clf_rbf.cv_results_['params']]]
temp = np.array(temp).transpose(1,0).tolist()
form = pd.DataFrame(temp,columns=['score','gamma'])
display(form)

Linear


,score,coef0,degree,gamma
0,0.824490,0.0100,2.0,0.01
1,0.942857,0.0100,2.0,0.16
2,0.926531,0.0100,2.0,0.50
3,0.824490,0.0100,5.0,0.01
4,0.942857,0.0100,5.0,0.16
5,0.926531,0.0100,5.0,0.50
6,0.824490,0.0100,10.0,0.01
7,0.942857,0.0100,10.0,0.16
8,0.926531,0.0100,10.0,0.50
9,0.824490,0.0015,2.0,0.01


RBF


,score,gamma
0,0.824490,0.01
1,0.942857,0.16
2,0.926531,0.50


# Comparison and Conclusion
我發現在 question 中，degree 對結果的影響不大，我想這是因為只要 degree 大到足以分類以後，再增加維度就並沒有明顯意義了  
除了 SVM，這次作業我學會了如何使用 sklearn 做到參數調整，希望在之後的 final project 可以有所幫助